## App ID Scraping
We're interested in knowing the app identifier (e.g. com.foobar.app) behind every application name mentioned in the dataset in order to reliably identify the app behind the transactions.

### Dataset Cleaning
We remove rows involving 'Play Points', we might consider them later.

In [ ]:
import pandas as pd

df = pd.read_json('unique_game_ids.json')
df.columns = ['app_name']

# Drop rows that contain 'Play Points' in their names
df = df[~df['app_name'].str.contains('Play Points', na=False)]

### App Search
We search for the app on the Play Store using its name and then return the closest reasonable match if there is any.

### Match Functions
We need functions that define, given a set of results, the closest match

In [ ]:
from google_play_scraper import search
from difflib import get_close_matches

# Too bad, apps often change their names
def get_first_app_id(app_name):
    results = search(app_name, lang="en", country="us")
    for result in results:
        if result['title'].lower() == app_name.lower():
            return result['appId']
    return None

# Uses fuzzy matching with a reasonable threshold, yields around 75% success rate!
def get_closest_app_id(app_name):
    try:
        results = search(app_name, lang="en", country="us")
        titles = [r['title'] for r in results]
        close_matches = get_close_matches(app_name, titles, n=1, cutoff=0.6)
        if close_matches:
            # Find the result that matches the best title
            match_title = close_matches[0]
            for result in results:
                if result['title'] == match_title:
                    return result['appId']
    except Exception as e:
        print(f"Error searching for {app_name}: {e}")
    return None

#### Single-Threaded Search Version (it takes 2h for our 11000 app names...)

In [ ]:
# Apply the fetch app ID function to each row
#from tqdm import tqdm
#tqdm.pandas()
#df['app_id'] = df['app_name'].progress_apply(get_closest_app_id)
#
## Compute ratio of successful matches
#total_apps = len(df)
#found_count = df['app_id'].notna().sum()
#success_ratio = found_count / total_apps
#
#print(f"Found {found_count} out of {total_apps} apps. Success ratio: {success_ratio:.2%}")

#### "Gotta Go Fast" Version (Google doesn't like it!! (But I use Mullvad to rotate proxies 😊🕳️🪏🤫))

##### Load Latest Mullvad Relays

In [ ]:
import requests

# Ask Mullvad for its relays
answer = requests.get("https://api.mullvad.net/www/relays/all/")
relays = answer.json()
# Only keep the ones that work over SOCKS
proxies = [
    f"socks5h://{relay['ipv4_addr_in']}:1080"
    for relay in relays
    if relay.get('socks_name') and relay.get('ipv4_addr_in')
]

print(f"Loaded {len(proxies)} SOCKS5 proxies from Mullvad.")

##### Inject Proxy Rotation Into The HTTP Get Function

In [ ]:
import random
import functools
import urllib

# Backup the original function
original_urlopen = urllib.request.urlopen

# Define the proxy-rotating function
def proxied_urlopen(request, *args, **kwargs):
    proxy = random.choice(proxies)
    print('Rotating to proxy on '+proxy)
    proxy_handler = urllib.request.ProxyHandler({
        'http': proxy,
        'https': proxy,
    })
    opener = urllib.request.build_opener(proxy_handler)
    return opener.open(request, *args, **kwargs)

# Monkey-patch the HTTP GET request function with the proxy-rotating one
urllib.request.urlopen = proxied_urlopen

##### Fetch App IDs in a Threaded Way

In [ ]:

from concurrent.futures import ThreadPoolExecutor, as_completed
from google_play_scraper import search
from difflib import get_close_matches
from tqdm import tqdm

# Use a tuple of (df_index, app_name)
def fetch_app_id_with_index(index, app_name):
    return index, get_closest_app_id(app_name)

def run_threaded_app_search(df, max_workers=10):
    results_dict = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(fetch_app_id_with_index, idx, row['app_name']): idx
            for idx, row in df.iterrows()
        }
        for future in tqdm(as_completed(futures), total=len(futures), desc="Searching apps"):
            idx, app_id = future.result()
            results_dict[idx] = app_id

    # Map results into a new column using original DataFrame index
    df['app_id'] = df.index.map(results_dict.get)
    return df

# Now run it safely
df = run_threaded_app_search(df)

### Save Dataset

In [ ]:
df.to_csv("OtherData/AppIdData/app_ids.csv", index=False)

### Reading The Saved Dataset

In [ ]:
df = pd.read_csv('OtherData/AppIdData/app_ids.csv')

# Compute ratio of successful matches
total_apps = len(df)
found_count = df['app_id'].notna().sum()
success_ratio = found_count / total_apps

print(f"Found {found_count} out of {total_apps} apps. Success ratio: {success_ratio:.2%}")

from IPython.display import display

# Remove truncation for wide/tall DataFrames (optional)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)         # Prevent wrapping
pd.set_option('display.max_colwidth', None)  # Show full column content
display(df) 